#  ZNIB Gravity model

In [1]:
from maa.config.utils import configure_logging, LogLevel
from maa.config.constants import CONFIGURATION_PATH
from maa.znib.znib import create_znib_gravity_models_from_config
import warnings
warnings.filterwarnings("ignore", module="statsmodels")
configure_logging(level=LogLevel.WARNING)
%load_ext autoreload
%autoreload 2

## Fit Gravity Models

In this step, two gravity models are fitted for each dataset—an **intra-institutional** and an **inter-institutional** model—using
the file paths and parameters defined in the configuration.

It constructs the link and graph datasets for both the **unfiltered (`CoAffAll`)** and **filtered (`CoAffStable`)** versions,
following the workflow outlined in the `create_co_affiliation_networks.ipynb` notebook.
From the set of unique affiliations, we generate the ZNIB model input by constructing all unique affiliation pairs, resulting in
a complete undirected network of `(n × n) / 2` possible edges.

These affiliation pairs are then enriched with:

- **Intra-organizational dummy variables**
  (e.g., `univ_univ`)
- **Inter-organizational dummy variables**
  (e.g., `univ_resi`)
- **Travel times** between affiliation locations

Finally, the enriched edge dataset is used to fit the **ZNIB gravity models**, separately for intra- and inter-institutional
relationships.

In [2]:
models = create_znib_gravity_models_from_config(config_path=CONFIGURATION_PATH)

2025-12-17T13:15:30.775814Z Dropped 246 affiliation link(s) from the DataFrame with no coordinates.


### Results for the intra-institutional proximity models

Across both datasets, greater travel time reduces the probability of co-affiliation links, confirming that geographical proximity remains a strong driver of simultaneous affiliations.
The variable ln_prod_edge_strength—the log product of the two institutions’ publication counts—is consistently positive and significant, indicating that larger institutions are more likely to be connected through co-affiliations simply because they host more researchers.

Institutional pairing effects reveal additional structural patterns. University–university (univ_univ) links show strong, highly significant positive effects in both datasets, highlighting universities’ central role as co-affiliation hubs.
Positive coefficients for medical–medical (med_med) and college–college (coll_coll) pairs appear in the CoAff–All dataset but weaken or disappear in CoAff–Stable. This reduction likely reflects the stricter filtering in the stable dataset, which removes many short-term or low-activity affiliations—common in medical and college settings—and the generally lower publication volume in these sectors.

In [3]:
models.all.znib_intra_model.summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                     ZeroInflatedNegativeBinomialP Regression Results                    
=========================================================================================
Dep. Variable:                                 y   No. Observations:              1027461
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                  1027451
Method:                                      MLE   Df Model:                            9
Date:                           Wed, 17 Dec 2025   Pseudo R-squ.:                  0.2369
Time:                                   14:36:12   Log-Likelihood:                -22388.
converged:                                  True   LL-Null:                       -29339.
Covariance Type:                             HC0   LLR p-value:                     0.000
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
inflate_const                     5.5929      0.157     35.642      0.000       5.285       5.900
inflate_ln_prod_article_count    -0.3275      0.011    -29.835      0.000      -0.349      -0.306
const                             1.0205      0.568      1.797      0.072      -0.093       2.134
ln_prod_article_count             0.2514      0.023     10.941      0.000       0.206       0.296
ln_duration                      -0.5690      0.053    -10.765      0.000      -0.673      -0.465
uni_uni                           2.5586      0.159     16.117      0.000       2.247       2.870
res_res                          -0.0635      0.245     -0.260      0.795      -0.543       0.416
med_med                           0.4016      0.176      2.284      0.022       0.057       0.746
comp_comp                        -0.4980      0.178     -2.805      0.005      -0.846      -0.150
coll_coll                         0.8230      0.335      2.456      0.014       0.166       1.480
npo_npo                           0.7061      0.383      1.846      0.065      -0.044       1.456
gov_gov                          -0.0053      0.996     -0.005      0.996      -1.957       1.946
alpha                            20.2041      1.067     18.928      0.000      18.112      22.296
=================================================================================================
"""

In [4]:
models.stable.znib_intra_model.summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                     ZeroInflatedNegativeBinomialP Regression Results                    
=========================================================================================
Dep. Variable:                                 y   No. Observations:                96141
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                    96131
Method:                                      MLE   Df Model:                            9
Date:                           Wed, 17 Dec 2025   Pseudo R-squ.:                  0.1631
Time:                                   14:36:13   Log-Likelihood:                -7293.6
converged:                                  True   LL-Null:                       -8715.3
Covariance Type:                             HC0   LLR p-value:                     0.000
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
inflate_const                     6.1487      0.222     27.673      0.000       5.713       6.584
inflate_ln_prod_article_count    -0.3513      0.014    -24.846      0.000      -0.379      -0.324
const                             2.8866      0.767      3.765      0.000       1.384       4.389
ln_prod_article_count             0.2280      0.031      7.360      0.000       0.167       0.289
ln_duration                      -0.5801      0.079     -7.307      0.000      -0.736      -0.424
uni_uni                           1.6615      0.201      8.252      0.000       1.267       2.056
res_res                          -0.6412      0.322     -1.989      0.047      -1.273      -0.009
med_med                          -0.2841      0.379     -0.749      0.454      -1.027       0.459
comp_comp                        -1.3610      0.451     -3.015      0.003      -2.246      -0.476
coll_coll                         0.7075      0.566      1.251      0.211      -0.401       1.816
npo_npo                           6.1145      5.434      1.125      0.261      -4.537      16.766
gov_gov                          -0.7558      1.671     -0.452      0.651      -4.030       2.518
alpha                            14.8225      2.277      6.510      0.000      10.360      19.285
=================================================================================================
"""

### Results for the inter-institutional proximity models

Two main patterns shape the count model results.
(1) Distance decay: higher travel time strongly reduces expected co-affiliation counts.
(2) Productivity effect: joint publication strength (ln prod_edge_strength) is positive and highly significant in CoAffAll, but becomes small and insignificant in CoAffStable, suggesting that productivity drives short-term or project-based co-affiliations but not persistent ones.

Institutional pairing effects show a clear hierarchy:

univ–resi is the only cross-type pairing with a positive, significant effect.

Most other cross-type pairs (gov, npo, med) are strongly negative—especially in the filtered network—indicating that these ties are rare and typically short-lived.

univ–coll is near zero in the full dataset and mildly negative in the filtered one.

Overall, filtering for temporal persistence removes short-term ties and sharpens structural differences across sectors: cross-type coefficients become more negative, reflecting the scarcity of durable inter-sector co-affiliations. The logit component shows that higher productivity reduces the probability of structural zeros, and the positive constant highlights the overall sparsity of inter-institutional co-affiliations.

In [5]:
models.all.znib_inter_model.summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                     ZeroInflatedNegativeBinomialP Regression Results                    
=========================================================================================
Dep. Variable:                                 y   No. Observations:              1027461
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                  1027437
Method:                                      MLE   Df Model:                           23
Date:                           Wed, 17 Dec 2025   Pseudo R-squ.:                  0.2535
Time:                                   14:36:13   Log-Likelihood:                -21903.
converged:                                  True   LL-Null:                       -29339.
Covariance Type:                             HC0   LLR p-value:                     0.000
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
inflate_const                     4.7356      0.118     40.167      0.000       4.505       4.967
inflate_ln_prod_article_count    -1.2908      0.038    -34.278      0.000      -1.365      -1.217
const                            -1.1633      0.207     -5.620      0.000      -1.569      -0.758
ln_prod_article_count             0.5925      0.057     10.321      0.000       0.480       0.705
ln_duration                      -0.8087      0.044    -18.476      0.000      -0.895      -0.723
coll_comp                        -0.9697      0.539     -1.800      0.072      -2.026       0.086
coll_gov                         -2.7638      0.493     -5.612      0.000      -3.729      -1.799
coll_med                         -2.5623      0.225    -11.406      0.000      -3.003      -2.122
coll_npo                         -1.5974      0.504     -3.166      0.002      -2.586      -0.609
coll_res                         -0.9721      0.304     -3.195      0.001      -1.569      -0.376
coll_uni                          0.0915      0.254      0.360      0.719      -0.407       0.590
comp_gov                         -2.9573      0.451     -6.553      0.000      -3.842      -2.073
comp_med                         -3.0287      0.412     -7.345      0.000      -3.837      -2.221
comp_npo                         -3.0901      0.341     -9.053      0.000      -3.759      -2.421
comp_res                         -2.3983      0.193    -12.403      0.000      -2.777      -2.019
comp_uni                         -1.6286      0.158    -10.324      0.000      -1.938      -1.319
gov_med                          -0.8179      0.427     -1.915      0.055      -1.655       0.019
gov_npo                          -1.9102      0.486     -3.928      0.000      -2.863      -0.957
gov_res                          -0.6581      0.594     -1.109      0.267      -1.821       0.505
gov_uni                          -0.8417      0.240     -3.500      0.000      -1.313      -0.370
med_npo                          -0.7620      0.210     -3.634      0.000      -1.173      -0.351
med_res                          -1.3093      0.223     -5.871      0.000      -1.746      -0.872
med_uni                          -0.6262      0.156     -4.019      0.000      -0.932      -0.321
npo_res                          -2.1727      0.486     -4.467      0.000      -3.126      -1.219
npo_uni                          -1.4103      0.234     -6.033      0.000      -1.868      -0.952
res_uni                           1.0526      0.253      4.160      0.000       0.557       1.549
alpha                            14.9386      0.566     26.381      0.000      13.829      16.048
=================================================================================================
"""

In [6]:
models.stable.znib_inter_model.summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                     ZeroInflatedNegativeBinomialP Regression Results                    
=========================================================================================
Dep. Variable:                                 y   No. Observations:                96141
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                    96117
Method:                                      MLE   Df Model:                           23
Date:                           Wed, 17 Dec 2025   Pseudo R-squ.:                  0.1777
Time:                                   14:36:14   Log-Likelihood:                -7166.7
converged:                                  True   LL-Null:                       -8715.3
Covariance Type:                             HC0   LLR p-value:                     0.000
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
inflate_const                     4.2986      0.199     21.612      0.000       3.909       4.688
inflate_ln_prod_article_count    -1.4067      0.057    -24.641      0.000      -1.519      -1.295
const                             1.3208      0.332      3.982      0.000       0.671       1.971
ln_prod_article_count             0.3063      0.099      3.089      0.002       0.112       0.501
ln_duration                      -0.7942      0.079    -10.047      0.000      -0.949      -0.639
coll_comp                        -0.8437      0.832     -1.014      0.310      -2.474       0.787
coll_gov                         -3.1286      0.766     -4.085      0.000      -4.630      -1.627
coll_med                         -8.2206      0.189    -43.473      0.000      -8.591      -7.850
coll_npo                         -2.7275      1.319     -2.069      0.039      -5.312      -0.143
coll_res                         -1.8808      0.531     -3.543      0.000      -2.921      -0.840
coll_uni                         -0.4086      0.350     -1.167      0.243      -1.095       0.278
comp_gov                         -3.6765      0.893     -4.117      0.000      -5.427      -1.926
comp_med                         -3.2935      0.715     -4.609      0.000      -4.694      -1.893
comp_npo                         -6.7238      0.821     -8.195      0.000      -8.332      -5.116
comp_res                         -2.8799      0.327     -8.801      0.000      -3.521      -2.239
comp_uni                         -1.8004      0.219     -8.221      0.000      -2.230      -1.371
gov_med                          -0.6627      0.729     -0.910      0.363      -2.091       0.765
gov_npo                          -2.7041      0.890     -3.039      0.002      -4.448      -0.960
gov_res                          -0.5781      0.787     -0.734      0.463      -2.121       0.965
gov_uni                          -1.1656      0.319     -3.649      0.000      -1.792      -0.539
med_npo                          -1.4731      0.455     -3.234      0.001      -2.366      -0.580
med_res                          -1.9189      0.325     -5.896      0.000      -2.557      -1.281
med_uni                          -0.9819      0.194     -5.062      0.000      -1.362      -0.602
npo_res                          -3.0280      0.798     -3.795      0.000      -4.592      -1.464
npo_uni                          -1.2948      0.382     -3.389      0.001      -2.044      -0.546
res_uni                           0.7473      0.312      2.396      0.017       0.136       1.358
alpha                            10.5876      1.891      5.598      0.000       6.881      14.294
=================================================================================================
"""